Üdv! Ez a Notebook Medgyes Csaba (RF8I8P) és Mészáros Bálint (HY90XY) projektmunkája a Globális Optimalizálás második részére. A projekt során az információt az alábbi forrásokból szereztük:

- https://medium.com/swlh/tabu-search-in-python-3199c44d44f1

- https://leeds-faculty.colorado.edu/glover/227%20-%20A%20Users%20Guide%20to%20Tabu%20Search%20-%20conversion%203%2026%2014.pdf

Mivel a Medium cikkben gyakorlatilag tökéletesen összefoglalták a módszert, így ezt szeretnénk demonstrálni, a végén három példával.

## Bevezetés

Szeretnénk betekintést tekinteni hogyan működik a TS módszer az NP nehéz Single Machine Total Weighted Tardiness Problem megoldásával.  

**The Single Machine Total Weighted Tardiness Problem**: Van egy gépünk mely egyszerre egy feladatot tud megoldani. Összesen $N$ feladat van melyeket megszakítás nélkül szeretnénk megoldani. Minden $i \in N$ feladatnak $P_i$ egész feldolgozási időre van szüksége, és pozitív súlya $W_i$ jelzi a feladat fontosságát és $d_i$ jelzi az esedékességét. Ha feltesszük, hogy a gép azonnal munkába áll, akkor az $i$ feladat befejezési idejét $C_i$-vel jelöljük, a munka késését pedig $T_i = max\{C_i − d_i, 0\}$ értékkel számíthatjuk ki. Ha a feladat feldolgozása a $C_i \leq d_i$ esedékessége előtt megtörténik, akkor nem lesz késés, tehát $T = 0$. A cél az, hogy a $N$ feladatokat úgy rendezzük, hogy minimálisra csökkenjen a teljes folyamat súlyozott késése, azaz $\min \sum W_i T_i$. Figyeljük meg, hogy ha egy munka nagyobb súlyú, a késés miatti büntetés magasabb lesz.

## Tabu Search

Először importáljuk a szükséges csomagokat:

In [14]:
import pandas as pd
import random as rd
from itertools import combinations
import math

Vegyük a következő példát és kezdjük a TS elméletével. Az alábbi módon van megadva az adatunk:

In [13]:
pd.read_excel('Instance_10.xlsx', names=['Job', 'weight', "processing_time", "due_date"],index_col=0).to_dict('index')

{1: {'weight': 0.5, 'processing_time': 1.5, 'due_date': 4},
 2: {'weight': 0.8, 'processing_time': 2.2, 'due_date': 4},
 3: {'weight': 0.6, 'processing_time': 0.5, 'due_date': 4},
 4: {'weight': 0.4, 'processing_time': 1.6, 'due_date': 6},
 5: {'weight': 0.4, 'processing_time': 3.5, 'due_date': 6},
 6: {'weight': 0.1, 'processing_time': 2.1, 'due_date': 6},
 7: {'weight': 0.2, 'processing_time': 2.5, 'due_date': 8},
 8: {'weight': 0.5, 'processing_time': 0.6, 'due_date': 8},
 9: {'weight': 0.3, 'processing_time': 3.2, 'due_date': 8},
 10: {'weight': 0.6, 'processing_time': 5.2, 'due_date': 8}}

Az optimalizálni kívánt célfüggvény $\min \sum W_i T_i$, melynek értéke az adott megoldás alkalmasságát jelöli. Most foglaljuk össze az algoritmust.

**Tabu Search Basic Algorithm**: A TS minden iterációban megpróbálja megtalálni a legjobb elfogadható megoldást az aktuális megoldás közelében, a legújabb megoldásokat „Tabu”-nak tekintve a ciklikusság megelőzésére. Kezdjük az algoritmus általános lépéseivel:

- **0. Lépés**: Egy kezdőmegoldás létrehozása. Ez az algoritmus kezdőpontja, gyakran randomizált.

- **1. lépés**: Tekintve, hogy megvan a kezdeti megoldás, a következő lépés az, hogy a jelenlegi $\mathbb{S}$ (jelen esetben a 0. iteráció kezdeti megoldása) megoldásból létrehozzuk a lehetséges megoldások listáját. Ezeket a megoldásokat hívjuk  $\mathbb{S}$ szomszédainak. Ahhoz, hogy a jelenlegi $\mathbb{S}$ megoldásból megtaláljuk a szomszédos megoldásokat, meg kell határoznunk a szomszédsági függvényt, amely minden $\matbb{S}$ megoldáshoz rendel egy megoldások részhalmazát.

Megjegyzés: Tegyük fel, hogy az SMTWTP jelenlegi megoldása $\mathbb{S}$ = [3, 8, 10, 4, 1, 6, 2, 5, 9, 7], és a szomszédsági függvény a swap, mely két feladat sorendjét cseréli ki. Ekkor az egyik szomszédsági megoldás a [8, 3, 10, 4, 1, 6, 2, 5, 9, 7] lenne. Ez $O(n^2)$, mivel $(n-2)$ szomszédja van az $n$ feladatból álló megoldásnak.

- **2. lépés**: Az első lépésben létrehozott szomszédsági listából válasszuk ki a legjobb nem-tabu vagy aspirációs kritériumnak megfelelő megoldást (lásd Diagram):

In [1]:
# import image module
from IPython.display import Image
  
# get the image
Image(url="tabuabra.jpg", width=400, height=400)

*Tabu List*: Ez az a lista, amellyel a TS rögzíti a legújabb megoldásokat, és bizonyos számú iterációig megakadályozza azok ismétlődését, ami segít a keresésnek eltávolodni a korábban látogatott megoldásoktól.

*Tabu Tenure*: Ez határozza meg a Tabu-lista méretét, vagyis azt, hogy hány iterációra maradjon meg egy megoldás-összetevő Tabuként.

Megyjegzés: problémánkban feltehetjük, hogy a Tabu Tenure 2, ekkor az első iterációban a Tabu List üres. Tegyük fel, hogy az első iterációban talált megengedett megoldás: [8, 3, 5, 4, 1, 6, 2, 10, 9, 7]. Ekkor a Tabu List:  {[8, 3, 5, 4, 1, 6, 2, 10, 9, 7]}, a második iterációban talált optimális megoldást még mindig: [8, 3, 5, 4, 1, 6, 2, 10, 9, 7], azonban ez nem megengedett, mert Tabu, szóval vesszük a következő legjobbat. Tegyük fel hogy ez: [8, 3, 5, 4, 1, 6, 2, 10, 7,9], ekkor a Tabu List: {[8, 3, 5, 4, 1, 6, 2, 10, 9, 7], [8, 3, 5, 4, 1, 6, 2, 10, 7,9]}. A harmadk iterációban [8, 3, 5, 4, 1, 6, 2, 10, 9, 7] megoldás elhagyja a Tabu listát, és a legjobb megengedett megoldás kerül a listába. Fontos megjegyzés, hogy minél kisebb a Tabu Tenure annál nagyobb a valószínűsége, hogy ciklikusság alakul ki.

*Tabu Attribute*: Ha nagy a probléma ($n$ száma magas) és a Tabu Tenure is magas, akkor érdmesebb a cseréket számon tartani, nem a teljes megoldásokat. Az előző példával élve, eltárolatjuk a cserélt munkákat, tehát a Tabu List {(8,3), (7,9)} alakú.

*Aspiration Criteria*: Ha az aktuális iterációban talált megoldás célfüggvény-értéke jobb, mint a jelenleg ismert legjobb megoldás értéke, de a lépés Tabu, akkor az Aspirációs Kritérium felülírja ennek a lépésnek Tabu állapotát, ezáltal az egyébként kizárt megoldásokat is belefoglalja az engedélyezett megoldások halmazába.

- **3. lépés**: Ellenőrizze a definiált leállítási feltételeket: ez lehet az elért iterációk maximális száma vagy a futás ideje. Ha a leállítási feltételek nem teljesülnek, menjen a 4. lépésre, ha a leállítási feltételek teljesülnek, fejezze be és adja vissza a legjobb megoldást.

- **4. lépés**: Frissítsük a Tabu listát, az aspirációs kritériumot és menjünk vissza az 1. lépésre.

### TS objektum kódja:

A következő TS osztály nagyjából ugyanaz mint a cikkben, azonban kommentekkel szeretnénk segíteni a megértést.

In [16]:
class TS():
    
    def __init__(self, Path, seed, tabu_tenure):
        
        self.Path = Path
        self.seed = seed
        self.tabu_tenure = tabu_tenure
        self.instance_dict = self.input_data()
        self.Initial_solution = self.get_InitialSolution()
        self.tabu_str, self.Best_solution, self.Best_objvalue = self.TSearch()
        
    def input_data(self):
        
        return pd.read_excel(self.Path, names=['Job', 'weight', "processing_time", "due_date"], index_col=0).to_dict('index')

    def get_tabuestructure(self):
        '''Takes a dict (input data)
        Returns a dict of tabu attributes(pair of jobs that are swapped) as keys and [tabu_time, MoveValue]
        '''
        dict = {}
        for swap in combinations(self.instance_dict.keys(), 2):
            dict[swap] = {'tabu_time': 0, 'MoveValue': 0}
        return dict

    def get_InitialSolution(self, show=False):
        n_jobs = len(self.instance_dict) # Number of jobs
        # Producing a random schedule of jobs
        initial_solution = list(range(1, n_jobs+1))
        rd.seed(self.seed)
        rd.shuffle(initial_solution)
        if show == True:
            print("initial Random Solution: {}".format(initial_solution))
        return initial_solution

    def Objfun(self, solution, show = False):
        '''Takes a set of scheduled jobs, dict (input data)
        Return the objective function value of the solution
        '''
        dict = self.instance_dict
        t = 0   #starting time
        objfun_value = 0
        for job in solution:
            C_i = t + dict[job]["processing_time"]  # Completion time
            d_i = dict[job]["due_date"]   # due date of the job
            T_i = max(0, C_i - d_i)    #tardiness for the job
            W_i = dict[job]["weight"]  # job's weight

            objfun_value +=  W_i * T_i
            t = C_i
        if show == True:
            print("\n","#"*8, "The Objective function value for {} solution schedule is: {}".format(solution ,objfun_value),"#"*8)
        return objfun_value

    def SwapMove(self, solution, i ,j):
        '''Takes a list (solution)
        returns a new neighbor solution with i, j swapped
       '''
        solution = solution.copy()
        # job index in the solution:
        i_index = solution.index(i)
        j_index = solution.index(j)
        #Swap
        solution[i_index], solution[j_index] = solution[j_index], solution[i_index]
        return solution

    def TSearch(self):
        '''The implementation Tabu search algorithm with short-term memory and pair_swap as Tabu attribute.
        '''
        # Parameters:
        tenure =self.tabu_tenure
        tabu_structure = self.get_tabuestructure()  # Initialize the data structures
        best_solution = self.Initial_solution
        best_objvalue = self.Objfun(best_solution)
        current_solution = self.Initial_solution
        current_objvalue = self.Objfun(current_solution)

        print("#"*30, "Short-term memory TS with Tabu Tenure: {}\nInitial Solution: {}, Initial Objvalue: {}".format(
            tenure, current_solution, current_objvalue), "#"*30, sep='\n\n')
        iter = 1
        Terminate = 0
        while Terminate < 100:
            print('\n\n### iter {}###  Current_Objvalue: {}, Best_Objvalue: {}'.format(iter, current_objvalue,
                                                                                    best_objvalue))
            # Searching the whole neighborhood of the current solution:
            for move in tabu_structure:
                candidate_solution = self.SwapMove(current_solution, move[0], move[1])
                candidate_objvalue = self.Objfun(candidate_solution)
                tabu_structure[move]['MoveValue'] = candidate_objvalue

            # Admissible move
            while True:
                # select the move with the lowest ObjValue in the neighborhood (minimization)
                best_move = min(tabu_structure, key =lambda x: tabu_structure[x]['MoveValue'])
                MoveValue = tabu_structure[best_move]["MoveValue"]
                tabu_time = tabu_structure[best_move]["tabu_time"]
                # Not Tabu
                if tabu_time < iter:
                    # make the move
                    current_solution = self.SwapMove(current_solution, best_move[0], best_move[1])
                    current_objvalue = self.Objfun(current_solution)
                    # Best Improving move
                    if MoveValue < best_objvalue:
                        best_solution = current_solution
                        best_objvalue = current_objvalue
                        print("   best_move: {}, Objvalue: {} => Best Improving => Admissible".format(best_move,
                                                                                                      current_objvalue))
                        Terminate = 0
                    else:
                        print("   ##Termination: {}## best_move: {}, Objvalue: {} => Least non-improving => "
                              "Admissible".format(Terminate,best_move,
                                                                                                           current_objvalue))
                        Terminate += 1
                    # update tabu_time for the move
                    tabu_structure[best_move]['tabu_time'] = iter + tenure
                    iter += 1
                    break
                # If tabu
                else:
                    # Aspiration
                    if MoveValue < best_objvalue:
                        # make the move
                        current_solution = self.SwapMove(current_solution, best_move[0], best_move[1])
                        current_objvalue = self.Objfun(current_solution)
                        best_solution = current_solution
                        best_objvalue = current_objvalue
                        print("   best_move: {}, Objvalue: {} => Aspiration => Admissible".format(best_move,
                                                                                                      current_objvalue))
                        Terminate = 0
                        iter += 1
                        break
                    else:
                        tabu_structure[best_move]["MoveValue"] = float('inf')
                        print("   best_move: {}, Objvalue: {} => Tabu => Inadmissible".format(best_move,
                                                                                              current_objvalue))
                        continue
        print('#'*50 , "Performed iterations: {}".format(iter), "Best found Solution: {} , Objvalue: {}".format(best_solution,best_objvalue), sep="\n")
        return tabu_structure, best_solution, best_objvalue

## Első teszt :

Az adatunk az alábbi:

In [20]:
pd.read_excel('Instance_10.xlsx', names=['Job', 'weight', "processing_time", "due_date"], index_col=0)

,weight,processing_time,due_date
Job,,,
1,0.5,1.5,4
2,0.8,2.2,4
3,0.6,0.5,4
4,0.4,1.6,6
5,0.4,3.5,6
6,0.1,2.1,6
7,0.2,2.5,8
8,0.5,0.6,8
9,0.3,3.2,8


Ekkor tesztelve a fentebbieket:

In [17]:
test = TS(Path="Instance_10.xlsx", seed = 2012, tabu_tenure=3)

##############################

Short-term memory TS with Tabu Tenure: 3
Initial Solution: [4, 7, 9, 1, 5, 3, 10, 6, 8, 2], Initial Objvalue: 39.080000000000005

##############################


### iter 1###  Current_Objvalue: 39.080000000000005, Best_Objvalue: 39.080000000000005
   best_move: (2, 7), Objvalue: 26.13 => Best Improving => Admissible


### iter 2###  Current_Objvalue: 26.13, Best_Objvalue: 26.13
   best_move: (8, 9), Objvalue: 17.930000000000003 => Best Improving => Admissible


### iter 3###  Current_Objvalue: 17.930000000000003, Best_Objvalue: 17.930000000000003
   best_move: (3, 4), Objvalue: 14.960000000000003 => Best Improving => Admissible


### iter 4###  Current_Objvalue: 14.960000000000003, Best_Objvalue: 14.960000000000003
   best_move: (4, 5), Objvalue: 14.200000000000003 => Best Improving => Admissible


### iter 5###  Current_Objvalue: 14.200000000000003, Best_Objvalue: 14.200000000000003
   best_move: (6, 7), Objvalue: 13.830000000000002 => Best Improving 

## Második teszt:

Az adatunk az alábbi:

In [21]:
pd.read_excel('Instance_20.xlsx', names=['Job', 'weight', "processing_time", "due_date"], index_col=0)

,weight,processing_time,due_date
Job,,,
1,0.5,0.5,8
2,0.8,3.6,6
3,0.6,2.8,5
4,0.4,1.9,12
5,0.4,1.0,10
6,0.1,1.8,11
7,0.2,2.6,6
8,0.5,4.7,18
9,0.3,1.8,8


Ekkor a teszt:

In [18]:
test = TS(Path="Instance_20.xlsx", seed = 2012, tabu_tenure=3)

##############################

Short-term memory TS with Tabu Tenure: 3
Initial Solution: [17, 14, 19, 18, 13, 20, 3, 2, 8, 6, 5, 15, 9, 1, 10, 11, 7, 12, 16, 4], Initial Objvalue: 70.75

##############################


### iter 1###  Current_Objvalue: 70.75, Best_Objvalue: 70.75
   best_move: (11, 17), Objvalue: 53.120000000000005 => Best Improving => Admissible


### iter 2###  Current_Objvalue: 53.120000000000005, Best_Objvalue: 53.120000000000005
   best_move: (1, 19), Objvalue: 41.94000000000002 => Best Improving => Admissible


### iter 3###  Current_Objvalue: 41.94000000000002, Best_Objvalue: 41.94000000000002
   best_move: (4, 8), Objvalue: 34.90000000000001 => Best Improving => Admissible


### iter 4###  Current_Objvalue: 34.90000000000001, Best_Objvalue: 34.90000000000001
   best_move: (10, 13), Objvalue: 28.36 => Best Improving => Admissible


### iter 5###  Current_Objvalue: 28.36, Best_Objvalue: 28.36
   best_move: (2, 10), Objvalue: 25.639999999999997 => Best Improving

## Harmadik teszt:

Az adatunk az alábbi:

In [22]:
pd.read_excel('Instance_30.xlsx', names=['Job', 'weight', "processing_time", "due_date"], index_col=0)

,weight,processing_time,due_date
Job,,,
1,0.5,6,29
2,0.9,1,38
3,0.0,2,28
4,0.3,6,35
5,0.6,5,43
6,0.3,4,44
7,0.9,6,26
8,0.4,1,18
9,0.2,2,48


Ekkor a teszt:

In [23]:
test = TS(Path="Instance_30.xlsx", seed = 2012, tabu_tenure=3)

##############################

Short-term memory TS with Tabu Tenure: 3
Initial Solution: [13, 2, 25, 24, 21, 22, 23, 6, 28, 27, 3, 5, 8, 18, 15, 14, 9, 30, 29, 26, 10, 20, 17, 1, 19, 11, 7, 12, 16, 4], Initial Objvalue: 479.4

##############################


### iter 1###  Current_Objvalue: 479.4, Best_Objvalue: 479.4
   best_move: (16, 25), Objvalue: 364.99999999999994 => Best Improving => Admissible


### iter 2###  Current_Objvalue: 364.99999999999994, Best_Objvalue: 364.99999999999994
   best_move: (11, 23), Objvalue: 297.19999999999993 => Best Improving => Admissible


### iter 3###  Current_Objvalue: 297.19999999999993, Best_Objvalue: 297.19999999999993
   best_move: (6, 7), Objvalue: 263.40000000000003 => Best Improving => Admissible


### iter 4###  Current_Objvalue: 263.40000000000003, Best_Objvalue: 263.40000000000003
   best_move: (13, 20), Objvalue: 239.39999999999998 => Best Improving => Admissible


### iter 5###  Current_Objvalue: 239.39999999999998, Best_Objvalue: 23